In [1]:
#Ass Mateus e Yan

# IMPORTAÇÕES
import numpy as np
import os, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import tensorflow as tf

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import StandardScaler 


from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog

In [40]:
#FUNÇÕES AUXILIARES


#Opção de ler a imagem colorida ou em escala de cinza
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
#    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))    
    return data

def show_cats_and_dogs(idx):
    cat = read_image(train_cats[idx])
    dog = read_image(train_dogs[idx])
    pair = np.concatenate((cat, dog), axis=1)
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()
    
# dois exemplos de descritores. Você deve criar outros mais robustos.
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):     
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #image = cv2.imread(image_file)        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [35]:
#NOSSOS DESCRITORES

#CNN igual ao do slide
def cnn_tf(image, image_size, image_nchannels):
    
    input_layer = tf.reshape(image, shape = [-1, image_size, image_size, image_nchannels])
    
    conv1 = tf.layers.conv2d(inputs = input_layer,
                             filters = 32,
                             kernel_size = [3,3],
                             padding = "same",
                             activation = tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs = conv1,
                                   pool_size = [2,2],
                                   strides = 2)
    
    conv2 = tf.layers.conv2d(inputs = pool1,
                             filters = 64,
                             kernel_size = [5,5],
                             padding = "same",
                             activation = tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                   pool_size = [2,2],
                                   strides = 2)
    
    flat = tf.contrib.layers.flatten(pool2)
    
#    return flat
    
    dense = tf.contrib.layers.fully_connected(inputs = flat,
                                             num_outputs = 128,
                                             activation_fn = tf.nn.relu)
    
    


In [28]:
#MONTANDO A BASE DE TREINAMENTO

#opção do número de imagens a serem usadas NIM

TRAIN_DIR = 'kaggle/train/'

ROWS = 128
COLS = 128
CHANNELS = 3
NIM = 1000

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] #full dataset: dogs and cats
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

# considera apenas NIM imagens. Para o dataset completo, desconsiderar.
train_images = train_dogs[:NIM] + train_cats[:NIM]
random.shuffle(train_images)

# Leitura das imagens
train = prep_data(train_images)
print("Train shape: {}".format(train.shape))

# Cria os labels (rótulos)
labels = []
for i in train_images:
    if 'dog' in i:
        labels.append(1)
    else:
        labels.append(0)


Processed 0 of 2000
Processed 250 of 2000
Processed 500 of 2000
Processed 750 of 2000
Processed 1000 of 2000
Processed 1250 of 2000
Processed 1500 of 2000
Processed 1750 of 2000
Train shape: (2000, 3, 128, 128)


In [7]:
#Apenas mostra algumas imagens do conjunto de treinamento


#for idx in range(0,3):
 #   show_cats_and_dogs(idx)

In [41]:
#Aqui passa cada imagem pelos descritores e salva nos vetores abaixo

rawImages = []
descHist = []
#cnnDesc = []

count = len(train_images)

for i, image_file in enumerate(train_images):
    image = read_image(image_file)
    pixels = image_to_feature_vector(image)
    histogram = extract_color_histogram(image)
    #neurons = cnn_tf(tf.cast(image, tf.float32), ROWS, CHANNELS)
    
    rawImages.append(pixels)
    descHist.append(histogram)
        
    if i%250 == 0: print('Processed {} of {}'.format(i, count))

Processed 0 of 2000
Processed 250 of 2000
Processed 500 of 2000
Processed 750 of 2000
Processed 1000 of 2000
Processed 1250 of 2000
Processed 1500 of 2000
Processed 1750 of 2000


In [25]:
#Avalia o primeiro descritor: as imagens raw
#Usa KNN, Arvore e Gaussian

(X_train, X_test, y_train, y_test) = train_test_split(
    rawImages, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))    

KNeighborsClassifier
****Results****
accuracy: 53.60%
DecisionTreeClassifier
****Results****
accuracy: 56.40%
GaussianNB
****Results****
accuracy: 58.00%


In [12]:
#Avalia o segundo descritor: color histogram
#Novamente com os três classificadores

(X_train, X_test, y_train, y_test) = train_test_split(
    descHist, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

KNeighborsClassifier
****Results****
accuracy: 57.20%
DecisionTreeClassifier
****Results****
accuracy: 62.80%
GaussianNB
****Results****
accuracy: 55.40%


In [13]:
#Avalia a combinação dos dois primeiros descritores!

#ATENÇÃO: ESTE É APENAS UM CÓDIGO EXEMPLO. VOCÊ DEVE DESENVOLVER
#DESCRITORES MAIS ROBUSTOS, BEM COMO EXPLORAR MELHOR AS MÉTRICAS
#DE AVALIAÇÃO (MATRIZ DE CONFUSÃO, ETC)

trainAux = np.hstack((descHist, rawImages))
(X_train, X_test, y_train, y_test) = train_test_split(
    trainAux, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

KNeighborsClassifier
****Results****
accuracy: 54.20%
DecisionTreeClassifier
****Results****
accuracy: 55.40%
GaussianNB
****Results****
accuracy: 58.60%


In [37]:
(X_train, X_test, y_train, y_test) = train_test_split(
    train_images, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]
   
for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

ValueError: Found input variables with inconsistent numbers of samples: [0, 2000]